## Continu Inzicht Demo (Real time)
> https://tscontinuinzicht.hkvservices.nl/ci-demo

### Deze notebook wordt gebruikt voor het vullen van Continu Inzicht Demo database
- Database aanpassen voor starten van berekening 
  - Tijdstip van momenten aanpassen
  - Achtergebleven data opruimen  
- Ophalen van waterstanden (belasting) van de meetstations
- Bepaal waterstand voor opgegeven momenten: [-24,0,24,48]
- Classificeer waterstand per meetstation
- Bepaal maatgegevende waterstand per dijkvak
- Bepaal technische faalkans voor een dijkvak
- Bepaal faalkans door maatregel voor een dijkvak
- Bepaal maatgevende faalkans voor een dijkvak
- Database aanpassen voor eindigen van berekening
  - Productie data verwijderen (data wat nu in UI te zien is)
  - Berekening data omzetten naar productie data
  - UI tijdstip aanpassen
  - Wijzigingen door UI resetten

**Imports**

In [ ]:
# import algemeen
from pathlib import Path

# Hiermee kan je markdown printen tijdens uitvoeren van functie
from IPython.display import Markdown, display

# imports base modules
from toolbox_continu_inzicht import Config, DataAdapter

**Inlezen configuratie**
<details>
<summary>Configuratie bekijken</summary>   

<pre style="font-size: 10pt">
GlobalVariables:
    rootdir: "data"
    calc_time: "2024-11-19"
    moments: [-24,0,24,48]

    LoadsWaterinfo:
            parameters: ["waterhoogte"]
            MISSING_VALUE: 999

    LoadsMatroos:
        website: "noos"
        model: "fews_rmm_km"
        parameters: ["WATHTE"]

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: ""
        parameters: [ "WATHTE [m][NAP][OW]" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false

    LoadsFewsLimburg:
        host: "http://109.232.44.231"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "App_Maas"
        parameters: [ "WATHTE" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false

    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: -999

DataAdapter:

    calculation_start_config:
        type: ci_postgresql_to_calculation_start
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"

    calculation_end_config:
        type: ci_postgresql_to_calculation_end
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"

    in_measuringstation_waterinfo:
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "waterinfo"

    in_measuringstation_matroos:
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "matroos"

    in_measuringstation_fews_hhnk:
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "fews_hhnk"

    in_measuringstation_fews_limburg:
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "fews_limburg"

    out_waterstanden_ci_postgresql_001:
        type: ci_postgresql_measuringstation_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 0.01

    out_waterstanden_ci_postgresql_1:
        type: ci_postgresql_measuringstation_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 1

    in_measuringstation_data_table:
        type: ci_postgresql_measuringstation_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime

    in_measuringstation_conditions_table:
        type: ci_postgresql_from_conditions
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"

    out_measuringstation_states_table:
        type: ci_postgresql_to_states
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"

    in_section_table:
        type: ci_postgresql_from_sections
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"

    in_sectionfractions_table:
        type: ci_postgresql_from_sectionfractions
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"

    out_waterstanden_section_ci_postgresql:
        type: ci_postgresql_section_load_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"        

    df_moment_waterstanden:
        type: python

    in_fragilitycurves_table:
        type: ci_postgresql_fragilitycurves_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        measureid: 0

    in_measure_fragilitycurves_table:
        type: ci_postgresql_measure_fragilitycurves_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime                

    in_section_load_from_data_table:
        type: ci_postgresql_section_load_from_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime

    out_section_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 5

    out_section_technical_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 100

    out_section_measure_failure_probability_data:
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 101        

    in_section_conditions:
        type: ci_postgresql_section_thresholds_from_conditions_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime

    in_section_data_failure_probability:
        type: ci_postgresql_section_failure_probability_from_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime

    out_section_states:
        type: ci_postgresql_section_to_states
        database: continuinzicht
        schema: continuinzicht_demo_realtime
</pre>
</details>

In [ ]:
# Lees configuratiebestand
yaml_config_file = "continu-inzicht-demo.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

**Aanmaken adapter**
<div style="width:40%; display:inline-table;">
    Algemene config:
    <pre style="font-size: 10pt;border:none">
    <b>GlobalVariables:</b>
        rootdir: "data"
        calc_time: "2024-11-19"
        moments: [-24,0,24,48]
    </pre>
<div>

In [ ]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

**Controleer of er al een berekening gaande is**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_ci_status:</b>
        type: ci_postgresql_calc_status
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "waterinfo"
    </pre>
</div>

In [ ]:
# De adapter 'ci_postgresql_calc_status' geeft de huidige rekenstatus en eventuele ui aanpassingen terug
df_in = data_adapter.input("in_ci_status")

# controleer het resultaat op is_calculating = True
is_calculating = df_in[df_in["is_calculating"]]
if len(is_calculating) > 0:
    display(Markdown("**⚠️ Er wordt al gerekend, toch doorgaan?**"))
    raise SystemExit("Er wordt al gerekend!")
else:
    display(Markdown("**✅ Er wordt niet gerekend**"))

**Start Continu Inzicht berekening**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>calculation_start_config:</b>
        type: ci_postgresql_to_calculation_start
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
<div>

In [ ]:
from toolbox_continu_inzicht.helpers import calculation_start

start, end = calculation_start(
    data_adapter=data_adapter, output="calculation_start_config", calc_time=None
)
print(f"{start} - {end}")

**Ophalen waterstanden via Waterinfo**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_waterinfo:</b>
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "waterinfo"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_waterstanden_ci_postgresql_001:</b>
        type: ci_postgresql_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"
        unit_conversion_factor: 0.01
    </pre>
</div>
<div style="width:40%; display:inline-table;">
    Functie config:
    <pre style="font-size: 10pt;border:none">
    <b>LoadsWaterinfo:</b>
        parameters: ["waterhoogte"]
        MISSING_VALUE: 999
    </pre>
</div>


In [ ]:
# Waterinfo waterstanden
# Vanuit WRIJ: vak 48-1_5, waarschijnlijk 100% gekoppeld aan Lobith – data uit RWS Waterinfo
# https://waterinfo.rws.nl/api/chart/get?mapType=waterhoogte&locationCodes=Lobith(LOBI)&values=-48%2C48

from toolbox_continu_inzicht.loads import LoadsWaterinfo

waterinfo = LoadsWaterinfo(data_adapter=data_adapter)

# try:
waterinfo.run(
    input="in_measuringstation_waterinfo",
    output="out_waterstanden_ci_postgresql_001",
)
df_out = waterinfo.df_out

# except Exception as e:
#     display(Markdown(f"**⚠️ {e}**"))

**Ophalen waterstanden via Noos Matroos**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_matroos:</b>
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "matroos"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_waterstanden_ci_postgresql_001:</b>
        type: ci_postgresql_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        unit_conversion_factor: 0.01
    </pre>
</div>
<div style="width:40%; display:inline-table;">
    Functie config:
    <pre style="font-size: 10pt;border:none">
    <b>LoadsMatroos:</b>
        website: "noos"
        model: "fews_rmm_km"
        parameters: ["WATHTE"]
    </pre>
</div>

In [ ]:
# Noos Matroos waterstanden
# Vanuit WSBD: vak 34-2_31, waarschijnlijk gekoppeld aan 2 stations vanuit NOOS Matroos (Hollandsch Diep km 993 en Hollandsch Diep km km 993)
# https://noos.matroos.rws.nl/direct/get_series.php?source=fews_rmm_km&unit=waterlevel&loc=Hollandsch%20Diep%20km%20994&anal_time=0000-00-00T00%3A00%3A00Z&tstart=2024-11-18T00%3A00%3A00.000Z&tstop=2024-11-22T00%3A00%3A00.000Z&format=dd

from datetime import datetime

from toolbox_continu_inzicht.loads import LoadsMatroos

try:
    # gebruik tijdelijk een locale versie van de data
    df_in = data_adapter.input("in_matroos_temp")
    df_in["date_time"] = df_in["date_time"].apply(lambda x: datetime.fromisoformat(x))
    data_adapter.output("out_waterstanden_ci_postgresql_001", df_in)
except FileNotFoundError:
    try:
        matroos = LoadsMatroos(data_adapter=data_adapter)
        matroos.run(
            input="in_measuringstation_matroos",
            output="out_waterstanden_ci_postgresql_001",
        )
        data_adapter.output("in_matroos_temp", matroos.df_out)

    except Exception as e:
        display(Markdown(f"**⚠️ {e}**"))

**Ophalen waterstanden via Fews (HHNK)**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_fews_hhnk:</b>
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "fews_hhnk"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_waterstanden_ci_postgresql_1:</b>
        type: ci_postgresql_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"
        unit_conversion_factor: 1
    </pre>
</div>
<div style="width:40%; display:inline-table;">
    Functie config:
    <pre style="font-size: 10pt;border:none">
    <b>LoadsFews:</b>
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: ""
        parameters: [ "WATHTE [m][NAP][OW]" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false<br>
    <b>GlobalVariables</b>
        aquo_allias:
            WATHTE [m][NAP][OW]: "WATHTE"
    </pre>
</div>

In [ ]:
# Fews HHNK waterstanden
# Vanuit HHNK: vak 6a gekoppeld aan FEWS HHNK locatie MPN-N-24
# https://fews.hhnk.nl/FewsWebServices/rest/fewspiservice/v1/timeseries?locationIds=MPN-N-24&parameterIds=WATHTE%20%5Bm%5D%5BNAP%5D%5BOW%5D
from toolbox_continu_inzicht.loads import LoadsFews

fews_hhnk = LoadsFews(data_adapter=data_adapter)

try:
    fews_hhnk.run(
        input="in_measuringstation_fews_hhnk", output="out_waterstanden_ci_postgresql_1"
    )
    df_out = fews_hhnk.df_out

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Ophalen waterstanden via Fews (Limburg)**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_fews_limburg:</b>
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "fews_limburg"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_waterstanden_ci_postgresql_1:</b>
        type: ci_postgresql_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"
        unit_conversion_factor: 1
    </pre>
</div>
<div style="width:40%; display:inline-table;">
    Functie config:
    <pre style="font-size: 10pt;border:none">
    <b>LoadsFews:</b>
        host: "http://109.232.44.231"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "App_Maas"
        parameters: [ "WATHTE" ]
        uurdata: true,
        addnearestmoments: true,
        addmaxmoment: false
    </pre>
</div>

In [ ]:
# Fews Limburg waterstanden
# Vanuit WL: vak 76-1_VAK037 gekoppeld Roermond boven als die bestaat in RWS Waterwebservices
# http://109.232.44.231/FewsWebServices/rest/fewspiservice/v1/timeseries?filterId=App_Maas&locationIds=Maas%20km%2079.00%2CMaas%20km%2080.00&parameterIds=WATHTE

# from toolbox_continu_inzicht.loads import LoadsFews
# fews_hhnk = LoadsFews(data_adapter=data_adapter)
# fews_hhnk.run(input="in_measuringstation_fews_limburg", output="out_waterstanden_ci_postgresql_1")
# df_out = fews_hhnk.df_out


**Ophalen waterstanden via RWS Waterwebservices**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_fews_limburg:</b>
        type: ci_postgresql_from_measuringstations
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        source: "fews_limburg"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_waterstanden_ci_postgresql_001:</b>
        type: ci_postgresql_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"
        unit_conversion_factor: 0.01
    </pre>
</div>
<div style="width:40%; display:inline-table;">
    Functie config:
    <pre style="font-size: 10pt;border:none">
    <b>LoadsWaterwebservicesRWS:</b>
        parameters: ["WATHTE"]
        MISSING_VALUE: -999
    </pre>
</div>

In [ ]:
# Fews Limburg waterstanden
# Vanuit WL: vak 76-1_VAK037 gekoppeld Roermond boven als die bestaat in RWS Waterwebservices
# http://109.232.44.231/FewsWebServices/rest/fewspiservice/v1/timeseries?filterId=App_Maas&locationIds=Maas%20km%2079.00%2CMaas%20km%2080.00&parameterIds=WATHTE

from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices = LoadsWaterwebservicesRWS(data_adapter=data_adapter)

try:
    waterwebservices.run(
        input="in_measuringstation_fews_limburg",
        output="out_waterstanden_ci_postgresql_001",
    )
    df_out = waterwebservices.df_out

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**van alle waterstanden alleen de waterstanden van opgegeven momenten halen**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_data_table:</b>
        type: ci_postgresql_measuringstation_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>df_moment_waterstanden:</b>
        type: python
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.loads import LoadsToMoments

# - waterstanden van de meetstations per moment ophalen
# - dataframe moments toevoegen aan adapter voor vervolg functies

load_moments = LoadsToMoments(data_adapter=data_adapter)
try:
    load_moments.run(
        input="in_measuringstation_data_table", output="df_moment_waterstanden"
    )

    if load_moments.df_out is not None:
        df_moments = load_moments.df_out.reset_index(drop=False)
        df_moments["date_time"] = df_moments["date_time"].astype(object)

        # dataframe moments toevoegen aan adapter
        data_adapter.set_dataframe_adapter("df_moment_waterstanden", df_moments)

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Status bepalen voor de meetstations**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measuringstation_conditions_table:</b>
        type: ci_postgresql_from_conditions
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>df_moment_waterstanden:</b>
        type: python
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_measuringstation_states_table:</b>
        type: ci_postgresql_to_states
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "measuringstation"
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.loads import LoadsClassify

# - via opgegeven waterstandsgrenzen wordt per meetstation de status bepaald

loads_classify = LoadsClassify(data_adapter=data_adapter)

try:
    loads_classify.run(
        input=["in_measuringstation_conditions_table", "df_moment_waterstanden"],
        output="out_measuringstation_states_table",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Waterstanden bepalen voor de dijkvakken**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_section_table:</b>
        type: ci_postgresql_from_sections
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>df_moment_waterstanden:</b>
        type: python
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>in_sectionfractions_table:</b>
        type: ci_postgresql_from_sectionfractions
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_waterstanden_section_ci_postgresql:</b>
        type: ci_postgresql_to_data
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        objecttype: "section"
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.sections import SectionsLoads

# - er wordt via de bovenliggende en onderliggende meetstation
#   en de fractie per meetstation (opgegeven in 'section fraction')
#   bepaald wat de maatgevende waterstand voor de dijkvak is.

if df_moments is not None:
    sections_loads = SectionsLoads(data_adapter=data_adapter)
    try:
        sections_loads.run(
            input=[
                "in_section_table",
                "df_moment_waterstanden",
                "in_sectionfractions_table",
            ],
            output="out_waterstanden_section_ci_postgresql",
        )

    except Exception as e:
        display(Markdown(f"**⚠️ {e}**"))

**Genereer fragility curves voor GEKB (Overloop en overslag)**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_section_slopes:</b>
        type: ci_postgresql_slopes
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>in_section_profiles:</b>
        type: ci_postgresql_profiles
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>in_section_bedlevel_fetch:</b>
        type: ci_postgresql_bedlevelfetch
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"    
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_fragility_curves:</b>
        type: ci_postgresql_fragilitycurves_table
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre> 
</div>
<div style="width:40%; display:inline-table;">
    Functie config:
    <pre style="font-size: 10pt;border:none">
    <b>FragilityCurveOvertoppingMultiple:</b>    
        gh_onz_mu: 0.96 # default waarde
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.fragility_curves import FragilityCurveOvertoppingMultiple

fragility_curve_overtopping = FragilityCurveOvertoppingMultiple(
    data_adapter=data_adapter
)

# try:
fragility_curve_overtopping.run(
    input=["in_section_slopes", "in_section_profiles", "in_section_bedlevel_fetch"],
    output="out_section_fragility_curves",
)

# except Exception as e:
#     display(Markdown(f"**⚠️ {e}**"))

**Genereer een gecombineerde fragility curves (COMB)**
<div style="width:40%; display:inline-table;">
    <pre style="font-size: 10pt;border:none">
    <b>fragility_curve_overloop_en_overslag:</b>
        type: input_ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "GEKB"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>fragility_curve_opbarsten_en_piping:</b>
        type: input_ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "STPH"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>fragility_curve_stabiliteit_binnenwaarts_dijken:</b>
        type: input_ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "STBI"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>fragility_curve_overloop_en_overslag_langsconstructies:</b>
        type: input_ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"        
        failuremechanism: "HTKW"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>fragility_curve_stabiliteit_langsconstructies:</b>
        type: input_ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"        
        failuremechanism: "STKKWI"
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>fragility_curve_piping_langsconstructies:</b>
        type: input_ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"        
        failuremechanism: "PKW"        
    </pre>        
    <pre style="font-size: 10pt;border:none">
    <b>out_section_fragility_curves:</b>
        type: ci_postgresql_fragilitycurves_table
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre> 
</div>

In [ ]:
from toolbox_continu_inzicht.fragility_curves import CombineFragilityCurvesIndependent

# Bij een aantal faalmchanismes is er voor bepaalde dijkvakken
# geen fragility curve aanwezig, dit is nu niet erg.
combine_fragility_curves = CombineFragilityCurvesIndependent(data_adapter=data_adapter)

try:
    combine_fragility_curves.run(
        input=[
            "fragility_curve_overloop_en_overslag",
            "fragility_curve_opbarsten_en_piping",
            "fragility_curve_stabiliteit_binnenwaarts_dijken",
            "fragility_curve_overloop_en_overslag_langsconstructies",
            "fragility_curve_stabiliteit_langsconstructies",
            "fragility_curve_piping_langsconstructies",
        ],
        output="out_section_fragility_curves",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Genereer een gecombineerde fragility curves (COMB)**
<div style="width:40%; display:inline-table;">
    <pre style="font-size: 10pt;border:none">
    <b>in_fragility_curve:</b>
        type: ci_postgresql_fragilitycurves
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>in_sections:</b>
        type: ci_postgresql_from_sections
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>    
    <pre style="font-size: 10pt;border:none">
    <b>in_failuremechanisms:</b>
        type: ci_postgresql_failuremechanisms
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>in_measures:</b>
        type: ci_postgresql_measures
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>df_combined_curves:</b>
        type: "python"  
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_fragility_curves:</b>
        type: ci_postgresql_fragilitycurves_table
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.fragility_curves import CombineFragilityCurvesIndependent

try:
    df_sections = data_adapter.input("in_sections")
    df_failuremechanisms = data_adapter.input("in_failuremechanisms")
    df_measures = data_adapter.input("in_measures")
    df_fragility_curve = data_adapter.input("in_fragility_curve")
    input_lists = []
    for _, row_section in df_sections.iterrows():
        for _, row_measure in df_measures.iterrows():
            input_list = []

            for _, row_failuremechanisms in df_failuremechanisms.iterrows():
                timedep = 0
                degradatie_id = 0
                section_id = row_section["id"]
                measure_id = row_measure["id"]
                failuremechanism_id = row_failuremechanisms["id"]
                failuremechanism_name = row_failuremechanisms["name"]
                if failuremechanism_name == "COMB":
                    pass
                else:
                    df = df_fragility_curve[
                        (df_fragility_curve["section_id"] == section_id)
                        & (
                            df_fragility_curve["failuremechanismid"]
                            == failuremechanism_id
                        )
                        & (df_fragility_curve["measureid"] == measure_id)
                        & (df_fragility_curve["timedep"] == timedep)
                        & (df_fragility_curve["degradatieid"] == degradatie_id)
                    ]

                    if len(df) > 0:
                        input_name = f"df_{failuremechanism_name}"
                        data_adapter.set_dataframe_adapter(
                            input_name, df, if_not_exist="create"
                        )
                        input_list.append(input_name)

                if len(input_list) > 0:
                    combine_fragility_curves = CombineFragilityCurvesIndependent(
                        data_adapter=data_adapter
                    )
                    combine_fragility_curves.run(
                        input=input_list,
                        output="df_combined_curves",
                    )

                    df_out = combine_fragility_curves.df_out

                    if df_out is not None:
                        df_out["section_id"] = section_id
                        df_out["measureid"] = measure_id
                        df_out["timedep"] = timedep
                        df_out["degradatieid"] = degradatie_id

                        # opslaan
                        data_adapter.output("out_section_fragility_curves", df_out)
                        input_lists.append((section_id, input_list))

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Bepaal technische faalkans voor de dijkvakken**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">    
    <b>in_fragilitycurves_table:</b>
        type: ci_postgresql_fragilitycurves_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        measureid: 0
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>in_section_load_from_data_table:</b>
        type: ci_postgresql_section_load_from_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_technical_failure_probability_data:</b>
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 100
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.sections import SectionsTechnicalFailureprobability

sections_failureprobability = SectionsTechnicalFailureprobability(
    data_adapter=data_adapter
)

try:
    sections_failureprobability.run(
        input=["in_fragilitycurves_table", "in_section_load_from_data_table"],
        output="out_section_technical_failure_probability_data",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Bepaal faalkans door maatregel voor de dijkvakken**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_measure_fragilitycurves_table:</b>
        type: ci_postgresql_measure_fragilitycurves_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime    
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>in_section_load_from_data_table:</b>
        type: ci_postgresql_section_load_from_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_measure_failure_probability_data:</b>
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 101 
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.sections import SectionsMeasureFailureprobability

sections_failureprobability = SectionsMeasureFailureprobability(
    data_adapter=data_adapter
)

try:
    sections_failureprobability.run(
        input=["in_measure_fragilitycurves_table", "in_section_load_from_data_table"],
        output="out_section_measure_failure_probability_data",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Bepaal faalkans door beheerdersoordeel voor de dijkvakken**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_expert_judgement_table:</b>    
        type: ci_postgresql_section_expert_judgement_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_expert_judgement_failure_probability_data:</b>
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 102
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.sections import SectionsExpertJudgementFailureprobability

sections_failureprobability = SectionsExpertJudgementFailureprobability(
    data_adapter=data_adapter
)

try:
    sections_failureprobability.run(
        input="in_expert_judgement_table",
        output="out_section_expert_judgement_failure_probability_data",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Bepaal de maatgevende faalkans**
1) beheerdersoordeel        (102)
2) faalkans door maatregel  (101)
3) technische faalkans      (100)
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_section_data_failure_probability:</b>
        type: ci_postgresql_section_failure_probability_from_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_failure_probability_data:</b>
        type: ci_postgresql_section_to_data
        database: continuinzicht
        schema: continuinzicht_demo_realtime
        parameter_id: 5
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.sections import SectionsFailureprobability

sections_failureprobability = SectionsFailureprobability(data_adapter=data_adapter)

try:
    sections_failureprobability.run(
        input="in_section_data_failure_probability",
        output="out_section_failure_probability_data",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Bepaal de status van de dijkvak voor alle berekende faalkansen**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>in_section_conditions:</b>
        type: ci_postgresql_section_thresholds_from_conditions_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>in_section_data_failure_probability:</b>
        type: ci_postgresql_section_failure_probability_from_data_table
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
    <pre style="font-size: 10pt;border:none">
    <b>out_section_states:</b>
        type: ci_postgresql_section_to_states
        database: continuinzicht
        schema: continuinzicht_demo_realtime
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.sections import SectionsClassify

sections_classify = SectionsClassify(data_adapter=data_adapter)
try:
    sections_classify.run(
        input=["in_section_conditions", "in_section_data_failure_probability"],
        output="out_section_states",
    )

except Exception as e:
    display(Markdown(f"**⚠️ {e}**"))

**Beeindig de berekening**
<div style="width:40%; display:inline-table;">
    Adapter config:
    <pre style="font-size: 10pt;border:none">
    <b>calculation_end_config:</b>
        type: ci_postgresql_to_calculation_end
        database: "continuinzicht"
        schema: "continuinzicht_demo_realtime"
    </pre>
</div>

In [ ]:
from toolbox_continu_inzicht.helpers import calculation_end

calculation_end(data_adapter=data_adapter, output="calculation_end_config")

In [ ]:
# in_ci_status:
#     type: ci_postgresql_calc_status
#     database: "continuinzicht"
#     schema: "continuinzicht_demo_realtime"

# df_in = data_adapter.input("in_ci_status")

is_calculating = df_in["is_calculating"]
len(is_calculating) > 0

is_calculating